In [9]:
class Data:
    
    def __init__(self, x=None, edge_index=None, edge_attr=None, y=None, **kwargs):
        r"""
        Args:
            x (Tensor, optional): 节点属性矩阵，大小为`[num_nodes, num_node_features]`
            edge_index (LongTensor, optional): 边索引矩阵，大小为`[2, num_edges]`，第0行为 尾节点，第1行为头节点，头指向尾
            edge_attr (Tensor, optional): 边属性矩阵，大小为`[num_edges, num_edge_features]`
            y (Tensor, optional): 节点或图的标签，任意大小(，其实也可以是边的标签)
        """
        self.x = x
        self.edge_index = edge_index
        self.edge_attr = edge_attr
        self.y = y
        for key, item in kwargs.items():
            if key == 'num_nodes':
                self.__num_nodes__ = item
            else:
                self[key] = item
                
class MyData(Data):
    def __init__(self, node_paper=None, node_institute=None, node_author=None,
                 edge_index_author_institute=None,edge_index_paper_author=None,y=None):  
        super(MyData,self).__init__(y=y)  
        self.node_paper = node_paper
        self.node_institute = node_institute
        self.node_author = node_author
        self.edge_index_author_institute = edge_index_author_institute
        self.edge_index_paper_author = edge_index_paper_author
    
    @property
    def node_paper_num(self):
        return self.node_paper.shape[0]
    
    @property
    def node_author_num(self):
        return self.node_author.shape[0]
    
    @property
    def node_institute_num(self):
        return self.node_institute.shape[0]